# EX14 Transformer Chatbot  
## Objective  
1. 한국어 전처리를 통해 학습 데이터셋을 구축하였다.  
- 공백과 특수문자 처리, 토크나이징, 병렬데이터 구축의 과정이 적절히 진행되었다.  
2. 트랜스포머 모델을 구현하여 한국어 챗봇 모델 학습을 정상적으로 진행하였다.  
- 구현한 트랜스포머 모델이 한국어 병렬 데이터 학습 시 안정적으로 수렴하였다.  
3. 한국어 입력문장에 대해 한국어로 답변하는 함수를 구현하였다.  
- 한국어 입력문장에 맥락에 맞는 한국어로 답변을 리턴하였다.  

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Step 1.  데이터 수집하기
한국어 챗봇 데이터는 송영숙님이 공개한 챗봇 데이터를 사용합니다  
https://github.com/songys/Chatbot_data   
일상다반사 0, 이별(부정) 1, 사랑(긍정) 2로 레이블링  

In [4]:
file_path=os.getenv('HOME') + '/aiffel/transformer_chatbot/data/ChatbotData .csv'
original_data = pd.read_csv(file_path)
original_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [13]:
len(original_data)

11823

총 11823개의 한국어 챗봇 데이터를 불러왔습니다

### Step 2. 데이터 전처리하기  
지난 요약 프로젝트에서 사용한 방법입니다  
- 중복 제거  
- 결측치 제거  
- 텍스트 정규화  
- 특수문자 제거  
- 다시한번 결측치 제거  
- 이상치 제거 (지나치게 긴 문장 등)  

In [9]:
#결측치확인
original_data.isna().sum()

Q        0
A        0
label    0
dtype: int64

In [11]:
#중복제거
original_data = original_data.drop_duplicates()
original_data.shape

(11823, 3)

결측치와 중복은 없었습니다  
정규화를 시켜줍니다  
- 문장에서 단어와 구두점 사이에 공백을 추가  
- 알파벳(한글)과 ! ? , . 이 4개의 구두점을 제외하고 다른 특수문자는 모두 제거

In [ ]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()


    # (한글, a-z, A-Z, ".", "?", "!", ",")를 제외한 모든 문자를 공백인 ' '로 대체합니다.
    sentence = re.sub(r"^[ㄱ-ㅎ|가-힣|a-z|A-Z|0-9|]+$", " ", sentence)

    # 단어와 구두점(punctuation) 사이의 거리를 만듭니다.
    # 예를 들어서 "I am a student." => "I am a student ."와 같이
    # student와 온점 사이에 거리를 만듭니다.
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = sentence.strip()
    return sentence

In [ ]:
data = original_data.copy()

### Step 3. SubwordTextEncoder 사용하기  
한국어 데이터는 형태소 분석기를 사용하여 토크나이징을 해야 한다고 많은 분이 알고 있습니다  
하지만 여기서는 형태소 분석기가 아닌 실습에서 사용했던 내부 단어 토크나이저인 SubwordTextEncoder를 그대로 사용해봅니다  

### Step 4. 모델 구성하기

### Step 5. 모델 평가하기
